## How to implement Heston model
In the other notebooks we used the `HestonMCModel` from `qablet.heston.mc`.
We show here how you could have created the same Heston model, using base MC modules in qablet. This example can also be used to make other modifications to the model, such as time dependent Heston Model, or even other kinds of volatility models.

Also see [Notebook 2.1 Custom MC](../../notebooks/2_1_custom_mc.ipynb) for more information on how to write a custom Monte Carlo model that can price qablet contracts.

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import SFC64, Generator
from qablet.base.mc import MCModel, MCStateBase
from qablet.base.utils import Forwards
from qablet.heston.mc import HestonMCModel
from qablet_contracts.eq.cliquet import Accumulator
from qablet_contracts.timetable import py_to_ts

In [ ]:
# Define a class for the state of a single asset Heston MC process
class MyState(MCStateBase):
    def __init__(self, timetable, dataset):
        """The advance method does the real work of the simulation. The __init__ method
        just makes the necessary parameters handy."""

        super().__init__(timetable, dataset)

        self.n = dataset["MC"]["PATHS"]

        # create a random number generator
        self.rng = Generator(SFC64(dataset["MC"]["SEED"]))

        self.asset = dataset["HESTON"]["ASSET"]
        self.asset_fwd = Forwards(dataset["ASSETS"][self.asset])
        self.spot = self.asset_fwd.forward(0)

        self.heston_params = (
            dataset["HESTON"]["LONG_VAR"],
            dataset["HESTON"]["VOL_OF_VAR"],
            dataset["HESTON"]["MEANREV"],
            dataset["HESTON"]["CORRELATION"],
        )

        # Initialize the processes x (log stock) and v (variance)
        self.x_vec = np.zeros(self.n)  #
        self.v_vec = np.full(self.n, dataset["HESTON"]["INITIAL_VAR"])

        self.cur_time = 0

    def advance(self, new_time):
        """Update x_vec, v_vec in place when we move simulation by time dt."""
        dt = new_time - self.cur_time
        if dt < 1e-10:
            return

        (theta, vvol, meanrev, corr) = self.heston_params
        fwd_rate = self.asset_fwd.rate(new_time, self.cur_time)

        # generate the wiener processes
        cov = [[dt, corr * dt], [corr * dt, dt]]
        dz_pair = self.rng.multivariate_normal([0, 0], cov, self.n).transpose()

        vol_vec = np.sqrt(np.maximum(0.0, self.v_vec))  # Floor

        # Update log stock process
        self.x_vec += (fwd_rate - vol_vec * vol_vec / 2.0) * dt
        self.x_vec += vol_vec * dz_pair[0]

        # update the variance process
        self.v_vec += (theta - self.v_vec) * meanrev * dt
        self.v_vec += vvol * vol_vec * dz_pair[1]

        self.cur_time = new_time

    def get_value(self, unit):
        """Return the value of the unit at the current time."""
        if unit == self.asset:
            return self.spot * np.exp(self.x_vec)
        else:
            return None


class MyModel(MCModel):
    def state_class(self):
        return MyState

### Market Data
The data is identical to the other notebooks in the Heston workshop.

In [ ]:
ticker = "SPX"
rate = 0.00  # or 0.03 for 3%
div = 0.00  # 0.01
spot = 100
pricing_dt = datetime(2023, 12, 31)

# flat array for discounts and forwards
tmax = 2.0
times = np.array([0.0, tmax])
rates = np.array([rate, rate])
fwds = spot * np.exp((rate - div) * times)

heston_dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(pricing_dt).value,
    "ASSETS": {
        "USD": ("ZERO_RATES", np.column_stack((times, rates))),
        ticker: ("FORWARDS", np.column_stack((times, fwds))),
    },
    "MC": {
        "PATHS": 100_000,
        "TIMESTEP": 1 / 100,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "MEANREV": 10,
        "VOL_OF_VAR": 1,
        "CORRELATION": -1.0,
    },
}

### Accumulator Cliquet
Define an Accmulator Cliquet and price it using the Heston model from the qablet package, and the one we created above.

In [ ]:
global_floor = 0.0
local_floor = -0.01
local_cap = 0.01
fix_dates = pd.bdate_range(pricing_dt, periods=13, freq="1ME")

contract = Accumulator(
    "USD", ticker, fix_dates, global_floor, local_floor, local_cap
).timetable()

heston_model = HestonMCModel()  # Model from qablet package
heston_price, _ = heston_model.price(contract, heston_dataset)
print(f"Heston (package): {100 * heston_price:.3f}")

my_model = MyModel()  # Model we defined above
my_heston_price, _ = my_model.price(contract, heston_dataset)
print(f"Heston (mymodel): {100 * my_heston_price:.3f}")